<a href="https://colab.research.google.com/github/micah-shull/LLMs/blob/main/LLM_048_huggingFace_SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install python-dotenv
# !pip install transformers
# !pip install huggingface_hub

In [ ]:
from transformers import pipeline
from huggingface_hub import login
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv("/content/HUGGINGFACE_HUB_TOKEN.env")

# Login using the token
login(token=os.environ["HUGGINGFACE_HUB_TOKEN"])

# Create your pipeline
classifier = pipeline("sentiment-analysis",
                      model="distilbert-base-uncased-finetuned-sst-2-english")
result = classifier("I'm really enjoying Hugging Face with a token!")
print(result)

# Run it on some text
result = classifier("I'm really enjoying learning Hugging Face!")
print(result)
result = classifier("I hate jogging!")
print(result)
result = classifier("I dont' care either way")
print(result)
result = classifier("meh")
print(result)
result = classifier("whatever you say.")
print(result)
result = classifier("you see awfully sure of yourself")
print(result)

Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.999873161315918}]
[{'label': 'POSITIVE', 'score': 0.9998546838760376}]
[{'label': 'NEGATIVE', 'score': 0.9937392473220825}]
[{'label': 'NEGATIVE', 'score': 0.9991015195846558}]
[{'label': 'POSITIVE', 'score': 0.9790390133857727}]
[{'label': 'POSITIVE', 'score': 0.967018723487854}]
[{'label': 'POSITIVE', 'score': 0.9966733455657959}]


## 🤔 Why Is the Model So Confident?

You're seeing results like:
```python
[{'label': 'NEGATIVE', 'score': 0.999}]
[{'label': 'POSITIVE', 'score': 0.996}]
```

Even for **neutral or sarcastic text** like:
- "meh"
- "whatever you say"
- "you seem awfully sure of yourself"

This seems... overly confident, right? Here's why 👇

---

## 🧠 What's Actually Happening

### 1. **The Model Was Trained for Binary Classification**
The model you're using:
```python
"distilbert-base-uncased-finetuned-sst-2-english"
```
…was trained on the **Stanford Sentiment Treebank v2 (SST-2)** dataset.

**SST-2 only includes:**
- `POSITIVE`
- `NEGATIVE`

There’s **no "neutral" class**, no sarcasm, no subtlety, no "mixed feelings."  
So the model **must choose** between just two buckets—even when the text is ambiguous or unopinionated.

---

### 2. **Softmax Always Picks a Winner**
The model outputs **raw scores** (called *logits*) for each label.

Then it applies the **softmax function**, which converts logits into **probabilities that always sum to 1.0**.

> Even if the model is unsure, it **still picks the "most likely" class** with a high confidence value—especially in a 2-class scenario.

---

### 3. **Pretraining Bias + Overfitting**
LLMs like BERT or DistilBERT are pretrained on huge corpora (Wikipedia, books, etc.) and then **fine-tuned** on small datasets like SST-2.

Fine-tuning on a limited dataset with polarized opinions can cause:
- Overconfidence on short or vague sentences
- Misclassification of sarcasm or nuance
- Poor generalization to real-world tones (like "meh" or "whatever")

---

## 🔍 Try This Yourself

Let’s inspect the **raw logits** instead of the processed output:

This will show you how it always leans toward a class even when unsure.



In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

text = "meh"
inputs = tokenizer(text, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
logits = outputs.logits
probs = torch.nn.functional.softmax(logits, dim=-1)
print("Logits:", logits)
print("Probabilities:", probs)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Logits: tensor([[-1.8109,  2.0331]])
Probabilities: tensor([[0.0210, 0.9790]])


## ✅ What You Can Do About It

- **Use a multi-class sentiment model** with a neutral category  
  → e.g., `cardiffnlp/twitter-roberta-base-sentiment`  
  (has Positive, Neutral, Negative)

- **Try zero-shot classification** if you want more nuanced control:
```python
classifier = pipeline("zero-shot-classification")
result = classifier(
    "meh",
    candidate_labels=["positive", "negative", "neutral", "sarcastic"]
)
print(result)
```

- **Train your own model** on more subtle or domain-specific sentiment examples.